In [1]:
# -*- coding: utf-8 -*-
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
from utils_net_inference import *
import scipy.stats

In [2]:
# Path to signals
path = './timeseries.txt'

In [3]:
# Domains ids
ids = ['0', '20', '22', '35', '38', '54', '61', '64', '6', '70', '74', '75']

In [4]:
# Parameters for the network inference
# Tau max
tau_max = 12
# FDR rate
q = 0.03

In [5]:
network_list, strength_list = net_inference_FDR(path,ids,tau_max,q)

In [6]:
network_list

[['20', '22', -1, 4, 1, -0.4761034816622015, -3904.281406571045],
 ['20', '54', 0, 1, 0, -0.306732434084665, -4794.043300568529],
 ['20', '61', -10, -3, -6, 0.40693550045107235, 4927.043679229831],
 ['20', '70', 3, 6, 4, 0.3591438501577918, 4971.025316838217],
 ['20', '74', -3, 3, 0, -0.658810209804914, -44994.300426043104],
 ['20', '75', -12, -5, -8, 0.4250177000215324, 9853.407813032309],
 ['22', '54', -2, 3, 0, 0.34931485882624314, 2406.280549927201],
 ['22', '61', -8, -1, -6, -0.34119019482853946, -1820.724873194928],
 ['22', '70', -3, 3, 0, -0.303521308318714, -1851.6277295055056],
 ['22', '74', -5, 1, -2, 0.5961123306321922, 17943.70983894848],
 ['22', '75', -12, -7, -11, -0.34838966013687456, -3559.849582490768],
 ['35', '54', 8, 8, 8, 0.23287601382991405, 1544.3072596373993],
 ['38', '54', 0, 6, 0, 0.2831577818142795, 2018.6154700520883],
 ['54', '74', -6, 0, -3, 0.5950186168591978, 34136.35786896345],
 ['54', '75', -1, 7, 3, 0.3670833257467904, 7148.806543181031],
 ['61', '74'

In [7]:
strength_list

[['0', 0],
 ['20', 73444.10194228303],
 ['22', 31486.473980637933],
 ['35', 1544.3072596373993],
 ['38', 2018.6154700520883],
 ['54', 52048.4109923297],
 ['61', 32000.2878824981],
 ['64', 0],
 ['6', 0],
 ['70', 25005.18390692323],
 ['74', 161864.0166081932],
 ['75', 55291.70850136917]]